# Load SAMS database

In [ ]:
filename = 'sa-0.db'
start_time = '2019-01-01'
end_time   = '2019-01-15'

## Using `sams-reporter-pandas` to get Pandas dataframe `df`

In [ ]:
# This is a workaround to import modules with "-" in the name 

tmp = __import__("sams-reporter-pandas")
globals().update(vars(tmp))

# ToDo: Refactor SAMSSoftwareAccountingDB class to take kwargs instead of args from argparse

class dummy(object):
    def __init__(self): pass
    
args = dummy()
args.begin = dateutil.parser.parse(start_time)
args.end = dateutil.parser.parse(end_time)
args.filename = filename
args.verbose = True
args.user, args.ignore_user = [], []
args.project, args.ignore_project = [], []
args.software, args.ignore_software = [], []

sdb = SAMSSoftwareAccountingDB(args)
df = sdb.get_pandas_data_frame()
print('done.')

## Visualization helpers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def take_n_largest(series, n=10, rest_label='other'):
    large = series.nlargest(n)
    small = series.drop(large.index)
    series = large
    series[rest_label] = small.sum()
    return series

def aggregate_df(df, groupby='name', agg_key='total_time', drop=[], take_n=False):
    df_agg = df.groupby([groupby]).agg({agg_key : 'sum'})
    s = df_agg[agg_key].sort_values()
    s = s.drop(drop)
    if take_n: 
        s = take_n_largest(s, n=take_n)
        s = s[::-1]
    return s

def plot_bar_graph(s, start_time, end_time):
    plt.figure(figsize=(12,6))
    plt.title(f'Usage from {start_time} to {end_time}.')
    y_pos = np.arange(len(s))
    plt.barh(y_pos, s.values)
    plt.yticks(y_pos, s.index)
    plt.xlabel('Core-hours')
    plt.grid(True, axis='x')
    plt.semilogx([],[])

# Total usage

In [ ]:
s = aggregate_df(
    df, groupby='name', agg_key='cpu_time', 
    drop=['system', 'user', 'group', 'impi', 'OpenMPI'], take_n=12)
plot_bar_graph(s, start_time, end_time)

s = aggregate_df(df, groupby='project', agg_key='cpu_time', take_n=25)
plot_bar_graph(s, start_time, end_time)

s = aggregate_df(df, groupby='username', agg_key='cpu_time', take_n=25)
plot_bar_graph(s, start_time, end_time)

# Project usage

In [ ]:
project = 'c3se2019-1-1'

pdf = df[df['project'] == project]
users = aggregate_df(pdf, groupby='username', agg_key='cpu_time', take_n=0)
software = aggregate_df(pdf, groupby='name', agg_key='cpu_time', take_n=12)

print(f'Core-hours per user from {start_time} to {end_time}\n')
print(users[::-1])
print(f'\nTotal usage by project "{project}": {users.sum():E} core-hours')

plot_bar_graph(users, start_time, end_time)
plot_bar_graph(software, start_time, end_time)